In [1]:
import sympy as sp

# Global symbols
t = sp.symbols('t', real=True)
x = sp.Function('x')(t)
u = sp.Function('u')(t)
lam = sp.Function('lam')(t)
C1, C2 = sp.symbols('C1 C2')  # For general constants

In [2]:
def define_hamiltonian(lagrangian_expr, dynamics_expr):
    return lagrangian_expr + lam * dynamics_expr

def solve_costate(H, t_f=None, lam_tf=None, general=False):
    dHdx = sp.diff(H, x)
    ode = sp.Eq(sp.diff(lam, t), -dHdx)
    if general:
        sol = sp.dsolve(ode, lam)
    else:
        sol = sp.dsolve(ode, lam, ics={lam.subs(t, t_f): lam_tf})
    return sol.rhs

def solve_optimal_control(H, lam_expr):
    dHdu = sp.diff(H, u)
    u_sol = sp.solve(sp.Eq(dHdu, 0), u)[0]
    return u_sol.subs(lam, lam_expr)

def solve_state(f_dyn, u_expr, x0=None, t0=None, general=False):
    f_subbed = f_dyn.subs(u, u_expr)
    ode = sp.Eq(sp.diff(x, t), f_subbed)
    if general:
        sol = sp.dsolve(ode, x)
    else:
        sol = sp.dsolve(ode, x, ics={x.subs(t, t0): x0})
    return sol.rhs

def enforce_boundary_conditions(x_expr, t_0, x_0, t_f, x_f):
    C = list(x_expr.free_symbols & {C1, C2})
    eqs = [
        sp.Eq(x_expr.subs(t, t_0), x_0),
        sp.Eq(x_expr.subs(t, t_f), x_f)
    ]
    sol = sp.solve(eqs, C, dict=True)[0]
    return x_expr.subs(sol)

# ---------------------------- Main Entry Function ----------------------------

def pontryagin_pipeline(L, f_dyn, x_cond, t_int, lam_tf=None):
    """
    Unified Pontryagin pipeline:
    - If x_cond = (x0, xf): uses two-point boundary value solution
    - If x_cond = scalar: uses x0 and lam_tf for single-boundary case
    """
    t_0, t_f = t_int

    # Single or two-point condition?
    is_two_point = isinstance(x_cond, (tuple, list)) and len(x_cond) == 2

    if is_two_point:
        x_0, x_f = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, general=True))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr_gen = sp.simplify(solve_state(f_dyn, u_expr, general=True))
        x_expr_final = sp.simplify(enforce_boundary_conditions(x_expr_gen, t_0, x_0, t_f, x_f))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x_general(t)': x_expr_gen,
            'x(t)': x_expr_final
        }

    else:
        x_0 = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, t_f, lam_tf, general=False))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr = sp.simplify(solve_state(f_dyn, u_expr, x_0, t_0, general=False))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x(t)': x_expr
        }

In [3]:
problems = [
    {
        "label": "Problem 1",
        "L_expr": 3*x - 5*u**2,
        "Dyn_expr": 4*u - 2,
        "x_cond": 1,
        "t_int": (0, 3),
        "lam_tf": 0
    },
    {
        "label": "Problem 2",
        "L_expr": -(5*x - 4*u - u**2),
        "Dyn_expr": x + u,
        "x_cond": 1,
        "t_int": (0, 1),
        "lam_tf": 0
    },
    {
        "label": "Problem 3",
        "L_expr": x + 2*u**2,
        "Dyn_expr": u,
        "x_cond": 0,
        "t_int": (0, 1),
        "lam_tf": 0
    }
]

for prob in problems:
    print("="*40)
    print(prob["label"])
    result = pontryagin_pipeline(
        prob["L_expr"], prob["Dyn_expr"], prob["x_cond"], prob["t_int"], prob["lam_tf"]
    )
    for key, val in result.items():
        print(f"{key}: {val}")
    print()


Problem 1
Hamiltonian: 2*(2*u(t) - 1)*lam(t) - 5*u(t)**2 + 3*x(t)
lambda(t): 9 - 3*t
u(t): 18/5 - 6*t/5
x(t): -12*t**2/5 + 62*t/5 + 1

Problem 2
Hamiltonian: (u(t) + x(t))*lam(t) + u(t)**2 + 4*u(t) - 5*x(t)
lambda(t): 5 - 5*exp(1 - t)
u(t): 5*exp(1 - t)/2 - 9/2
x(t): (((-14 + 5*E)*exp(t) + 18)*exp(t - 1) - 5)*exp(1 - t)/4

Problem 3
Hamiltonian: lam(t)*u(t) + 2*u(t)**2 + x(t)
lambda(t): 1 - t
u(t): t/4 - 1/4
x(t): t*(t - 2)/8

